### Run this in Python kernel

To reset font style of output (in case it is not the default one)

In [1]:
%%html
<style>
body {
    font-family: "Arial", cursive, sans-serif; # font style of application
}
pre {
    font-family: "Courier New"; # font style of cell outputs
}
</style>  

### Run below in the backtesting_clojure kernel

In [20]:
; import libraries from kernel
(ns clojure-backtesting.demo
  (:require [clojure.test :refer :all]
            [oz.notebook.clojupyter :as oz]
            [clojure-backtesting.data :refer :all]
            [clojure-backtesting.order :refer :all]
            [clojure-backtesting.evaluate :refer :all]
            [clojure-backtesting.plot :refer :all]
            [clojure-backtesting.counter :refer :all]
            [clojure-backtesting.parameters :refer :all]
            [clojure.string :as str]
            [clojure.pprint :as pprint]
            [java-time :as t]
            [clojupyter.kernel.version :as ver]
            [clojupyter.misc.helper :as helper]
  ) ;; require all libriaries from core
  (:use clojure.pprint)
)

nil

### Import dataset

In [21]:
; path to dataset = "../resources/CRSP-extract.csv"
; change it to the relative to your own dataset
;
(reset! data-set (add-aprc (read-csv-row "../resources/CRSP-extract.csv")));

### Initialise portfolio

In [22]:
;; initialise with current date and initial capital (= $10000)
(init-portfolio "1980-12-16" 10000);

In [23]:
(keys (deref available-tics))

("AAPL" "F" "IBM")

### Write a strategy

The following code implements a simple trading strategy:

In a timespan of 10 days (inclusive of today),
- Buy 50 stocks of AAPL on the first day
- Sell 10 stocks of AAPL on every other day

In [24]:
;; define the "time span", i.e. to trade in the coming 10 days 
(def num-of-days (atom 10))                              

(while (pos? @num-of-days) ;; check if num-of-days is > 0
    (do 
        ;; write your trading strategy here
        (if (= 10 @num-of-days) ;; check if num-of-days == 10
            (do
                (order "AAPL" 50) ; buy 50 stocks
                (println ((fn [date] (str "Buy 50 stocks of AAPL on " date)) (get-date)))
            )
        )
        (if (odd? @num-of-days) ;; check if num-of-days is odd
            (do
                (order "AAPL" -10) ; sell 10 stocks
                (println ((fn [date] (str "Sell 10 stocks of AAPL on " date)) (get-date)))
            )
        )
        
        (update-eval-report (get-date)) ;; update the evaluation metrics every day
        
        ; move on to the next trading day
        (next-date)
        
        ; decrement counter
        (swap! num-of-days dec)
    )
)

; check whether counter == 0
(println ((fn [counter] (str "Counter: " counter)) @num-of-days))

Buy 50 stocks of AAPL on 1980-12-16
Sell 10 stocks of AAPL on 1980-12-17
Sell 10 stocks of AAPL on 1980-12-19
Sell 10 stocks of AAPL on 1980-12-23
Sell 10 stocks of AAPL on 1980-12-26
Sell 10 stocks of AAPL on 1980-12-30
Counter: 0


nil

### Check order record

In [25]:
(pprint/print-table (deref order-record))


|      :date | :tic |  :price | :aprc | :quantity |
|------------+------+---------+-------+-----------|
| 1980-12-17 | AAPL | 25.9375 | 26.09 |        50 |
| 1980-12-18 | AAPL | 26.6875 | 26.42 |       -10 |
| 1980-12-22 | AAPL | 29.6875 | 27.67 |       -10 |
| 1980-12-24 | AAPL | 32.5625 | 28.80 |       -10 |
| 1980-12-29 | AAPL | 36.0625 | 30.11 |       -10 |
| 1980-12-31 | AAPL | 34.1875 | 29.42 |       -10 |


nil

### Check portfolio record

In [26]:
;; view final portfolio
(view-portfolio)


| :asset |  :price | :aprc | :quantity | :tot-val |
|--------+---------+-------+-----------+----------|
|   cash |     N/A |   N/A |       N/A |    10295 |
|   AAPL | 34.1875 | 29.42 |         0 |        0 |


nil

In [27]:
;; view portfolio value and return
(view-portfolio-record)


|      :date | :tot-value | :daily-ret | :tot-ret | :loan | :leverage |
|------------+------------+------------+----------+-------+-----------|
| 1980-12-16 |     $10000 |      0.00% |    0.00% | $0.00 |     0.00% |
| 1980-12-17 |     $10007 |      0.00% |    0.08% | $0.00 |     0.00% |
| 1980-12-18 |     $10026 |      0.00% |    0.27% | $0.00 |     0.00% |
| 1980-12-19 |     $10054 |      0.27% |    0.54% | $0.00 |     0.00% |
| 1980-12-22 |     $10096 |      0.00% |    0.96% | $0.00 |     0.00% |
| 1980-12-23 |     $10111 |      0.15% |    1.11% | $0.00 |     0.00% |
| 1980-12-24 |     $10168 |      0.00% |    1.67% | $0.00 |     0.00% |
| 1980-12-26 |     $10191 |      0.22% |    1.89% | $0.00 |     0.00% |
| 1980-12-29 |     $10254 |      0.00% |    2.51% | $0.00 |     0.00% |
| 1980-12-30 |     $10251 |     -0.03% |    2.48% | $0.00 |     0.00% |
| 1980-12-31 |     $10295 |      0.00% |    2.91% | $0.00 |     0.00% |


nil

### Generate evaluation report

In [28]:
(eval-report)


|      :date | :pnl-pt | :sharpe | :tot-val |  :vol |
|------------+---------+---------+----------+-------|
| 1980-12-16 |      $7 |   1.41% |   $10007 | 0.06% |
| 1980-12-17 |     $13 |   2.45% |   $10026 | 0.11% |
| 1980-12-18 |     $13 |   0.00% |   $10026 | 0.00% |
| 1980-12-19 |     $32 |   4.86% |   $10096 | 0.20% |
| 1980-12-22 |     $32 |   7.88% |   $10096 | 0.12% |
| 1980-12-23 |     $42 |   7.86% |   $10168 | 0.21% |
| 1980-12-24 |     $42 |  15.31% |   $10168 | 0.11% |
| 1980-12-26 |     $50 |  11.99% |   $10254 | 0.21% |
| 1980-12-29 |     $50 |  22.48% |   $10254 | 0.11% |
| 1980-12-30 |     $49 |  19.10% |   $10295 | 0.15% |


nil

### Plot variables
Below are example codes that show how to plot different variables in the portfolio record / evaluation record.

### 1. Portfolio daily return

In [29]:
(def data (deref portfolio-value))

#'clojure-backtesting.demo/data

In [30]:
; Add legend name to series
(def data-to-plot
 (map #(assoc % :plot "port-value")
  data))

#'clojure-backtesting.demo/data-to-plot

In [31]:
(first data-to-plot)

{:date "1980-12-16", :tot-value 10000, :daily-ret 0.0, :tot-ret 0.0, :loan 0.0, :leverage 0.0, :plot "port-value"}

In [32]:
(plot data-to-plot :plot :date :daily-ret false)

### 2. Plot volatility

In [39]:
(def data (deref eval-record))

#'clojure-backtesting.demo/data

In [40]:
(first data)

{:date "1980-12-16", :pnl-pt 7.806415917975755, :sharpe 1.4142135623730954, :tot-val 10007.806415917976, :vol 0.05517816194058409}

In [41]:
; Add legend name to series
(def data-to-plot
 (map #(assoc % :plot "volatility")
  data))

#'clojure-backtesting.demo/data-to-plot

In [42]:
(eval-report)


|      :date | :pnl-pt | :sharpe | :tot-val |  :vol |
|------------+---------+---------+----------+-------|
| 1980-12-16 |      $7 |   1.41% |   $10007 | 0.06% |
| 1980-12-17 |     $13 |   2.45% |   $10026 | 0.11% |
| 1980-12-18 |     $13 |   0.00% |   $10026 | 0.00% |
| 1980-12-19 |     $32 |   4.86% |   $10096 | 0.20% |
| 1980-12-22 |     $32 |   7.88% |   $10096 | 0.12% |
| 1980-12-23 |     $42 |   7.86% |   $10168 | 0.21% |
| 1980-12-24 |     $42 |  15.31% |   $10168 | 0.11% |
| 1980-12-26 |     $50 |  11.99% |   $10254 | 0.21% |
| 1980-12-29 |     $50 |  22.48% |   $10254 | 0.11% |
| 1980-12-30 |     $49 |  19.10% |   $10295 | 0.15% |


nil

In [44]:
(plot data-to-plot :plot :date :vol true)

### 3. Plot sharpe ratio

In [46]:
; Add rename legend
(def data-to-plot
 (map #(assoc % :plot "sharpe ratio")
  data))

(plot data-to-plot :plot :date :sharpe true)

### 4. Plot stock price

In [47]:
(def data (deref order-record))

#'clojure-backtesting.demo/data

In [48]:
(first data)

{:date "1980-12-17", :tic "AAPL", :price 25.9375, :aprc "26.09", :quantity 50}

In [49]:
; Add rename legend
(def data-to-plot
 (map #(assoc % :plot "price")
  data))

(plot data-to-plot :plot :date :price true)